# Solving the Simple Pendulum with Velocity-Verlet: Lab 3 Solutions

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def rhs_pend(x):
    """
    rhs_pend: Evaluate the right hand side of the coupled (non-
        dimensional) ODEs describing the nonlinear pendulum
    
    ---INPUT:
    x - the current value of the dependent variable. For the pendulum
        ODEs x = [theta omega] where theta is the angle and omega is the
        angular velocity.
        
    ---OUTPUT:
    rhs - a row vector representing the value of the right hand side
        of the ODEs. Specifically, rhs=[omega -4*pi^2*sin(theta)].
    -------------------------------------------------------------------------------
    """
    
    rhs = np.zeros(2)
    
    theta = x[0];
    omega = x[1];
    rhs[0] = omega;
    rhs[1] = -4*np.pi**2*np.sin(theta)
    
    return rhs

In [4]:
def kepler_analytic(vel,T):

    #-------------------------------------------------------------------------------
    # Calculate the analytic trajectory for the Kepler central force problem.
    # Assumes an initial position r = (1,0), v = (0,vel).
    #-------------------------------------------------------------------------------
    # INPUTS:
    # - vel: the initial speed.
    # - T: sets an upper limit for |theta| for the e > 1 case (plot up to the
    #        integration time).
    #-------------------------------------------------------------------------------

    # Calculate trajectory from analytic solution

    ecc = np.linalg.norm(vel)**2 - 1        # Eccentricity
    a = 1/(1 - ecc)                         # Semi-major axis

    if ecc < 1:

        theta = np.linspace(0, 2*np.pi, 50) # Equally spaced values from 0 to 2*pi
        b = a * np.sqrt(1 - ecc**2)
        xan = -a * ecc + a * np.cos(theta)
        yan =  b * np.sin(theta)

    else:

        b = a * np.sqrt(ecc**2 - 1)
        theta_max = np.asinh(np.linsalg.norm(vel) * T/b) # Limit for range of theta
        that = np.linspace( -theta_max, theta_max, 50)
        xan = - a * ecc + a * np.cosh(theta)
        yan =   b * np.sinh(theta)

    return xan, yan

## <span style="color:red">Q1</span>
### Modify the code so that it also plots $\theta$ versus $t.$ Run the code and produce $\theta$ versus $t$ plots for numerical integrations from $t=0$ to $t=7$ with time steps $\tau=0.01,\,0.1,\,0.2,\,1,$ and with initial angles $\theta_1=10,\,45,\,90,\,170\,{\rm deg}.$

For which time steps, and which initial angles, is the numerical solution *reasonably accurate*?

We find accurate solution for $\tau$ = 0.01, 0.2, but inaccurate solutions for $\tau$ = 0.2, 1.

## <span style="color:red">Q2</span>
### There is some subjectivity in how you deﬁne whether a numerical solution is “reasonably accurate”.

Which of the following could be a valid way of determining whether a numerical solution of pendulum motion is accurate? 

## <span style="color:red">Q3</span>
### Velocity-Verlet Method for Pendulum Dynamics

Velocity-Verlet is the variant of the Verlet method introduced in Lecture 2. For the general dynamics problem Velocity-Verlet may be written:

$${\bf r}_{n+1}={\bf r}_n+\tau {\bf v}_{n}+\frac{1}{2}\tau^2{\bf a}_n$$

and

$$ {\bf v}_{n+1}= {\bf v}_n+\tfrac{1}{2}\tau\left({\bf a}_n+{\bf a}_{n+1}\right).$$

Using your RK4 method as a starting point, create new Python code which solves the pendulum problem using Velocity-Verlet instead of fourth-order Runge-Kutta.

*Hints*: Note that, for the pendulum ODEs the correspondence with the dynamics equations for 1-D motion with position $x$ and velocity $v$ is:

$$ x\quad \Leftrightarrow \quad \theta, $$
$$v=\frac{dx}{dt}\quad \Leftrightarrow \quad \omega=\frac{d\theta}{dt}, $$
$$a=\frac{dv}{dt}\quad \Leftrightarrow \quad \alpha = \frac{d\omega}{dt}=-4\pi^2\sin\theta.$$

Hence the Velocity-Verlet updates for the pendulum are:

$$ \theta_{n+1}=\theta_n+\tau \omega_{n}+\tfrac{1}{2}\tau^2\alpha_n$$

and

$$\omega_{n+1}= \omega_n+\frac{1}{2}\tau\left(\alpha_n+\alpha_{n+1}\right),$$

where $\alpha_n=-4\pi^2\sin\theta_n.$

When you have your Velocity-Verlet code for the pendulum problem working, repeat the calculations from 1, placing an A (accurate) or I (inaccurate) in each place in the table below:

 $\theta_1=10$ deg $\theta_1=45$ deg $\theta_1=90$ deg $\theta_1=170$ deg $\tau=0.01$        
$\tau=0.1$
       $\tau=0.2$         $\tau=1$        

Compare your table to the one you generated above for RK4. You will notice that there is a one entry that is accurate for VV but inaccurate for RK4:
Compared to RK4 (which gave an inaccurate solution), the VV method is more accurate for $\tau = 0.2$.

Compared to RK4, we find slightly more accurate solutions for $\tau = 0.2$, particularly for low initial angles, $\theta_1$: VV maintains a more constant amplitude (and hence energy) than RK4 (although the period is more distorted relative to the RK4 solution). As an example, Figure 2 shows the results for the different time steps for $\theta_1 = 90°$. The case $\tau = 0.2$ has an increasing amplitude with time, and the case $\tau = 1$ has a rapidly increasing amplitude.

## <span style="color:red">Q4</span>
### We know that RK4 is more accurate than Velocity-Verlet in each single step, but that how much of an advantage does it give it for this simple pendulum problem?

# Solving the Kepler problem with RK4
## <span style="color:red">Q5</span>

### How well does RK4 perform on the circular motion test case (initial vector ${\bf x}=[1,0,0,1]$), with the default time step ($\tau=0.05$)?

The integration appears to be quite accurate for the circular motion test case with the default time step, τ = 0.05: the analytic trajectory is followed closely, and the variation in energy is small (≈ $10^{-7}$).

## <span style="color:red">Q6</span>

### Apply the RK4 scheme to simulate a highly eccentric case (v₁ = 0.3).
 
Step through orders of magnitude (e.g., τ = 0.05, 0.005, ...) to identify an estimate of the largest time step, τ, which yields a reasonably accurate orbit.

In [24]:
tau_values = [0.00005, 0.0005, 0.005, 0.05] 

# Set up figure 
fig, ax = plt.subplots(1,4, figsize=(15,4))

for i in range(4):
    
    
    pos_x, pos_y, time, energy = rk4_kepler(tau_values[i], 0.3)

    ax[i].plot(pos_x, pos_y, '--g', lw=3, label="RK4 method" if i==3 else None)
    ax[i].plot(0, 0, 'ko', markersize=10, label="Center" if i==3 else None)
    ax[i].tick_params(labelsize=15)
    ax[i].set_ylim(-2, 2)
    ax[i].set_xlabel("x position", fontsize=18)
    ax[i].set_title(f"tau = {tau_values[i]}")
    

ax[0].set_ylabel("y position", fontsize=18)
ax[3].legend(fontsize=10)


## <span style="color:red">Q7</span>

Apply the RK4 scheme to simulate a highly eccentric initial condition, $\mathbf{x}=[1,0,0,0.3]$, from Question 6.


## <span style="color:red">Q8</span>

Summarize the comparison of RK4 to the Verlet method on the Kepler problem.

## <span style="color:red">Answer</span>

The two methods have different advantages: RK4 is a bit more accurate for a given time step,
but Verlet conserves energy over an orbit.